<a href="https://colab.research.google.com/github/caldeirav/FinSynthAgent/blob/main/FinSynth_Agent_Concept_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FinSynth.ai

The AI agent, "FinSynth AI", specializes in reading, interpreting, and summarizing corporate filings (10-K/10-Q reports) and recent financial information via news / financial data APIs to provide a structured research that can be leveraged as reference to generate actionable insights for investment decision-making. This is an example of Agentic system leveraging the [smolagents](https://github.com/huggingface/smolagents) library from HuggingFace to demonstrate the orchestration capabilities of CodeAgents.


## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:the dependencies and login to our HF account to access the Inference API


In [ ]:
!pip install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 678.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Creating a custom Knowledge Base Tool with 10-K and 10-Q SEC filings

For specialized tasks, a custom knowledge base can be invaluable. Let's create a tool that queries a vector database of relevant Corporate Filings. Using semantic search, the agent will then be able to find the most relevant information to support its financial research needs.

This approach combines predefined knowledge with semantic search to provide context-aware solutions for financial information retrieval in official SEC filings. With specialized knowledge access, our FinSynth knowledge can incorporate references to audited financial data and corporate analysis information issued to shareholders.

Install the dependecies first and run!

In [ ]:
!pip install pandas pypdf tqdm langchain langchain-core langchain-community langchain-chroma langchain_huggingface langchain-text-splitters sentence-transformers --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s et

Now we prepare the knowledge base by processing the PDF documents and storing them into a vector database to be used by the retriever.

In [ ]:
import os
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import LocalAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer

## Extract PDF Files from ./data/ directory
pdf_directory = "./data"
pdf_files = [
    os.path.join(pdf_directory, f)
    for f in os.listdir(pdf_directory)
    if f.endswith(".pdf")
    ]
source_docs = []

for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    source_docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)


print("Embedding documents... This should take a few minutes")
# Initialize embeddings and ChromaDB vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(docs_processed, embeddings, persist_directory="./chroma_db")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Now the database is ready: let’s build our agentic RAG system! We  need a RetrieverTool that our agent can leverage to retrieve information from the knowledge base.

In [ ]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore

class RetrieverTool(Tool):
    name = "retriever"
    description = (
        "Uses semantic search to retrieve the parts of the relevant Form 10-K document (an annual report required by the U.S. Securities and Exchange Commission) that could be most relevant to answer your query."
    )
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vector_store: Chroma, **kwargs):
        super().__init__(**kwargs)
        self.vector_store = vector_store

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.vector_store.similarity_search(query, k=5)
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

As our document database is made of annual and quarterly report, we may not have the latest information on the company that is relevant for certain questions. So we also provide a web search tool to our agent.

In [ ]:
from smolagents import DuckDuckGoSearchTool

# Initialize the search tool
web_search_tool = DuckDuckGoSearchTool()

Now it’s straightforward to create an agent that leverages these tools. The agent will need these arguments upon initialization:

*   Tools: a list of tools that the agent will be able to call.
*   Model: the LLM that powers the agent.


Our model must be a callable that takes as input a list of messages and returns text. It also needs to accept a stop_sequences argument that indicates when to stop its generation. For convenience, we directly use the HfApiModel class provided in the package to get a LLM engine that calls our Inference API.

And we use the default model Qwen2.5-Coder-32B-Instruct, served for free on Hugging Face's Inference API.

In [ ]:
from smolagents import HfApiModel, CodeAgent

model = HfApiModel()

retriever_tool = RetrieverTool(vector_store)
agent = CodeAgent(
    tools=[retriever_tool, web_search_tool],
    model=model,
    planning_interval=1,
    max_steps=5,
    verbosity_level=2
)

In [ ]:
agent.visualize()

CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever    │ Uses semantic search to retrieve the parts of │ query (`string`): The query to perform. This │
    │              │ the relevant Form 10-K document (an annual    │ should be semantically close to your target  │
    │              │ report required by the U.S. Securities and    │ documents. Use the affirmative form rather   │
    │              │ Exchange Commission) that could be most       │ than a question.                             │
    │              │ relevant to answer your query.                │                                              │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

Since we initialized the agent as a ReactJsonAgent, it has been automatically given a default system prompt that tells the LLM engine to process step-by-step and generate tool calls as JSON blobs (you could replace this prompt template with your own as needed).

Then when its .run() method is launched, the agent takes care of calling the LLM engine, parsing the tool call JSON blobs and executing these tool calls, all in a loop that ends only when the final answer is provided.

In [ ]:
agent_output = agent.run("Provide a summary and description of ongoing or potential lawsuits NVIDIA is facing, including those disclosed in the annual report and recent developments.")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Provide a summary and description of ongoing or potential lawsuits NVIDIA is facing, including those disclosed  │
│ in the annual report and recent developments.                                                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To provide a summary and description of ongoing or potential lawsuits NVIDIA is facing, I'll first use the
`retriever` tool to search the NVIDIA Form 10-K document for related information. After that, I'll use the         
`web_search` tool to find any recent developments not mentioned in the annual report.                              
Code:                                                                                                              
```py                                                                                                              
nvidia_litigation = retriever(query="NVIDIA ongoing or potential lawsuits")                                        
print(nvidia_litigation)                                                                                           
```<end_code>                                                                                                      
                                                                                                                   
Observation:                                                                                                       
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_litigation = retriever(query="NVIDIA ongoing or potential lawsuits")                                      
  print(nvidia_litigation)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
procedures.
The putative derivative actions initially ﬁled September 24, 2019 and pending in the United States District Court 
for the District of
Delaware, Lipchitz v. Huang, et al. (Case No. 1:19-cv-01795-UNA) and Nelson v. Huang, et. al. (Case No. 
1:19-cv-01798- UNA),
remain stayed pending resolution of the plaintiffs’ appeal in the In Re NVIDIA Corporation Securities Litigation 
action. Following the
Ninth Circuit’s denial of NVIDIA’s petition for rehearing on November 15, 2023, the parties are conferring 
regarding the next steps in
these derivative matters. The lawsuits assert claims, purportedly on behalf of us, against certain ofﬁcers and 
directors of the
Company for breach of ﬁduciary duty, unjust enrichment, insider trading, misappropriation of information, corporate
waste and
violations of Sections 14(a), 10(b), and 20(a) of the Exchange Act based on the dissemination of allegedly false, 
and misleading
statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The plaintiffs seek 
unspeciﬁed
damages and other relief, including disgorgement of proﬁts from the sale of NVIDIA stock and unspeciﬁed corporate 
governance
measures.
Another putative derivative action was ﬁled on October 30, 2023 in the Court of Chancery of the State of Delaware, 
captioned
Horanic v. Huang, et al. (Case No. 2023-1096-KSJM). This lawsuit asserts claims, purportedly on behalf of us, 
against certain
ofﬁcers and directors of the Company for breach of ﬁduciary duty and insider trading based on the dissemination of 
allegedly false
and misleading statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The 
plaintiffs
seek unspeciﬁed damages and other relief, including disgorgement of proﬁts from the sale of NVIDIA stock and reform
of

===== Document 1 =====
Table of Contents
NVIDIA Corporation and Subsidiaries
Notes to the Consolidated Financial Statements
(Continued)
resolution of the matter. On December 5, 2023, the Ninth Circuit granted NVIDIA’s motion to stay the mandate. 
NVIDIA’s deadline to
ﬁle a petition for a writ of certiorari is March 4, 2024.
The putative derivative lawsuit pending in the United States District Court for the Northern District of 
California, captioned 4:19-cv-
00341-HSG, initially ﬁled January 18, 2019 and titled In re NVIDIA Corporation Consolidated Derivative Litigation, 
was stayed
pending resolution of the plaintiffs’ appeal in the In Re NVIDIA Corporation Securities Litigation action. On 
February 22, 2022, the
court administratively closed the case, but stated that it would reopen the case once the appeal in the In Re 
NVIDIA Corporation
Securities Litigation action is resolved. Following the Ninth Circuit’s denial of NVIDIA’s petition for rehearing 
on November 15, 2023,
the parties are conferring regarding the next steps in this derivative matter. The lawsuit asserts claims, 
purportedly on behalf of us,
against certain ofﬁcers and directors of the Company for breach of ﬁduciary duty, unjust enrichment, waste of 
corporate assets, and
violations of Sections 14(a), 10(b), and 20(a) of the Exchange Act based on the dissemination of allegedly false 
and misleading
statements related to channel inventory and the impact of cryptocurrency mining on GPU demand. The plaintiffs are 
seeking
unspeciﬁed damages and other relief, including reforms and improvements to NVIDIA’s corporate governance and 
internal
procedures.
The putative derivative actions initially ﬁled September 24, 2019 and pending in the United States District Court 
for the District of

===== Document 2 =====
United States District Court for the Northern District of California, and titled In Re NVIDIA Corporation 
Securities Litigation, ﬁled an
amended complaint on May 13, 2020. The amended complaint asserted that NVIDIA and certain NVIDIA executives 
violated Section
10(b) of the Securities Exchange Act of 1934, 

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Based on the information gathered from the NVIDIA Form 10-K document and the web search results, I can    
summarize the ongoing or potential lawsuits NVIDIA is facing, including recent developments.                       
                                                                                                                   
Here's a summary:                                                                                                  
1. **Derivative Lawsuits**: Several derivative lawsuits have been filed against NVIDIA and its officers and        
directors, alleging breaches of fiduciary duty, unjust enrichment, insider trading, and violations of the Exchange 
Act. These lawsuits are rooted in alleged false statements about channel inventory and the impact of cryptocurrency
mining on GPU demand.                                                                                              
2. **In Re NVIDIA Corporation Securities Litigation**: A significant securities class-action lawsuit was filed in  
2018, alleging that NVIDIA executives made materially false statements about channel inventory and cryptocurrency  
mining. This lawsuit was initially dismissed but has been revived by a Ninth Circuit ruling. The Supreme Court has 
dismissed NVIDIA's appeal, allowing the class-action lawsuit to proceed.                                           
3. **US Supreme Court Decision**: The US Supreme Court decided to allow a $1 billion class-action lawsuit to       
proceed against NVIDIA. This lawsuit accuses NVIDIA of securities fraud related to its dependence on cryptocurrency
mining.                                                                                                            
4. **Potential Antitrust Lawsuit**: There is potential for an antitrust lawsuit from the Justice Department against
NVIDIA. While NVIDIA denies being served[

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(                                                                                                    
      """                                                                                                          
  Summary of NVIDIA's ongoing or potential lawsuits:                                                               
                                                                                                                   
  1. Derivative lawsuits filed against NVIDIA and its officers/directors for alleged false statements about        
  channel inventory and cryptocurrency mining.                                                                     
  2. A significant securities class-action lawsuit, revived by a Ninth Circuit ruling and approved by the Supreme  
  Court, is seeking billions in damages from NVIDIA for securities fraud.                                          
  3. An ongoing antitrust investigation by the Justice Department, with NVIDIA denying being served a subpoena,    
  may result in a potential antitrust lawsuit.                                                                     
  """                                                                                                              
  )                                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Summary of NVIDIA's ongoing or potential lawsuits:

1. Derivative lawsuits filed against NVIDIA and its officers/directors for alleged false statements about channel 
inventory and cryptocurrency mining.
2. A significant securities class-action lawsuit, revived by a Ninth Circuit ruling and approved by the Supreme 
Court, is seeking billions in damages from NVIDIA for securities fraud.
3. An ongoing antitrust investigation by the Justice Department, with NVIDIA denying being served a subpoena, may 
result in a potential antitrust lawsuit.

[Step 2: Duration 40.06 seconds| Input tokens: 11,948 | Output tokens: 558]

Final output:

Summary of NVIDIA's ongoing or potential lawsuits:

1. Derivative lawsuits filed against NVIDIA and its officers/directors for alleged false statements about channel inventory and cryptocurrency mining.
2. A significant securities class-action lawsuit, revived by a Ninth Circuit ruling and approved by the Supreme Court, is seeking billions in damages from NVIDIA for securities fraud.
3. An ongoing antitrust investigation by the Justice Department, with NVIDIA denying being served a subpoena, may result in a potential antitrust lawsuit.

